# Exploratory Data Analysis

**Owner: Daniel Soukup - Created: 2025.11.01**

The goal of this notebook is to explore the dataset, understand our target column and features (statistical properties, data quality) and plan for preprocessing and modelling.

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np

import plotly.express as px

import plotly.offline as pyo
pyo.init_notebook_mode()

## Data Loading

Let's ensure the datasets are available:

In [0]:
client = dataiku.api_client()
project = client.get_project('US_CENSUS_PROJECT')

datasets = project.list_datasets()

for dataset in datasets:
    print(dataset["name"])

Load the datasets:

In [0]:
def load_data_by_name(name: str) -> pd.DataFrame:
    """
    Load dataset by its name.
    """
    mydataset = dataiku.Dataset(name)
    mydataset_df = mydataset.get_dataframe()
    
    return mydataset_df

train_df = load_data_by_name("census_income_learn")
test_df = load_data_by_name("census_income_test")

In [0]:
train_df.shape, test_df.shape

We have ~200K rows for testing and ~100K rows for testing with 42 columns, the last being our target. Note that we are missing the column names.

In [0]:
train_df.head()

In [0]:
test_df.head()

In [0]:
train_df.info()

**Observations:**

It is clear that we need to do a fair bit of cleaning:
- clarify the missing column names by matching the fields to the provided data dictionary
- there are some missing values in col11 that need to be investigated and addressed
- we have a high number of object data types that will need processing prior to modeling

Excerpt from the data dict:

- Number of instances data = 199523
    - Duplicate or conflicting instances : 46716
- Number of instances in test = 99762
    - Duplicate or conflicting instances : 20936
- Class probabilities for income-projected.test file
    - Probability for the label '- 50000' : 93.80%
    - Probability for the label '50000+' : 6.20%
- Number of attributes = 40 (continuous : 7 nominal : 33)

A quick check confirms that there are no exact duplicate columns per se:

In [0]:
# note: runs long
train_df.T.duplicated().sum()

## Column Mapping

Lets look at the high level stats first:

In [0]:
# numeric columns
train_df.describe()

In [0]:
# categorical
train_df.select_dtypes("object").describe()

We'll use the data dictionary and number of unique values to map our columns to their proper names. We also need this information to make well-informed decisions on how the columns should be processed for modelling.

In [0]:
pd.set_option('display.max_colwidth', 100) 

unique_values = pd.DataFrame(
    {
        "unique_values": [train_df[col].unique() for col in train_df.columns],
        "num_unique": [train_df[col].nunique() for col in train_df.columns],
        "dtype": [train_df[col].dtype for col in train_df.columns],
    },
    index=train_df.columns,
)
unique_values

Based on this we put together our mapping:

In [0]:
col_mapping = {
    "col_0": "age", # matches type and range
    "col_1": "class of worker", # unique values checked with data dict (UVDD)
    "col_2": "detailed industry recode", # UVDD
    "col_3": "detailed occupation recode", # UVDD
    "col_4": "education", # UVDD
    "col_5": "wage per hour", # looks to be at right position, type checks, in cents?
    "col_6": "enroll in edu inst last wk", # UVDD
    "col_7": "marital stat", # UVDD
    "col_8": "major industry code", # UVDD
    "col_9": "major occupation code", # UVDD
    "col_10": "race", # UVDD
    "col_11": "hispanic origin", # UVDD - 10 unique in data dict? values match though
    "col_12": "sex", # UVDD
    "col_13": "member of a labor union", # UVDD
    "col_14": "reason for unemployment", # UVDD
    "col_15": "full or part time employment stat", # UVDD
    "col_16": "capital gains", # data dict check, range ok, dollars?
    "col_17": "capital losses", # data dict check, range ok, dollars?
    "col_18": "dividends from stocks", # data dict check
    "col_19": "tax filer stat", # UVDD
    "col_20": "region of previous residence", # UVDD
    "col_21": "state of previous residence", # UVDD
    "col_22": "detailed household and family stat", # data dict check
    "col_23": "detailed household summary in household", # data dict check
    "col_24": "instance weight", # SPECIAL
    "col_25": "migration code-change in msa", # UVDD
    "col_26": "migration code-change in reg", # UVDD
    "col_27": "migration code-move within reg", # UVDD
    "col_28": "live in this house 1 year ago",# UVDD
    "col_29": "migration prev res in sunbelt", # UVDD
    "col_30": "num persons worked for employer", # value check
    "col_31": "family members under 18", # UVDD
    "col_32": "country of birth mother",  # UVDD
    "col_33": "country of birth self",  # UVDD
    "col_34": "country of birth father",  # UVDD
    "col_35": "citizenship", # UVDD
    "col_36": "own business or self employed", # UVDD
    "col_37": "fill inc questionnaire for veteran's admin", # UVDD
    "col_38": "veterans benefits", # UVDD
    "col_39": "weeks worked in year", # data dict order
    "col_40": "year", # UVDD
    "col_41": "income"
}

About the instance weight:

> The instance weight indicates the number of people in the population
 that each record represents due to stratified sampling.
 To do real analysis and derive conclusions, this field must be used.
 This attribute should *not* be used in the classifiers, so it is
 set to "ignore" in this file.
 
 
More info from [census.gov](https://www.census.gov/programs-surveys/cps/technical-documentation/methodology/weighting.html):

>The base weight, which is the inverse of the probability of the person being in the sample, is a rough measure of the number of actual persons that the sample person represents. Almost all sample persons in the same state have the same base weight, but the weights across states are different. Selection probabilities may also differ for some sample areas due to field subsampling, which is done when areas selected for the sample contain many more households than expected. The base weights are then adjusted for noninterview, and the ratio estimation procedure is applied.

We won't be using this field for modeling as per the data dictionary recommendation.

We are ready to map the names:

In [0]:
train_df = train_df.rename(columns=col_mapping)
test_df = test_df.rename(columns=col_mapping)

In [0]:
train_df.head()

## Univariate distributions

We look at the univariate distribution of each column that will help us deciding on any preprocessing.

### Numerical Fields

First, look at the numeric fields:

In [0]:
train_df.select_dtypes(['int', 'float'])

By a quick glance, we can recognize that some of these columns are codes actually hence better dealt with as categories:

In [0]:
num_only_df = train_df.select_dtypes(["int", "float"])

unique_values = pd.DataFrame(
    {
        "num_unique": [num_only_df[col].nunique() for col in num_only_df.columns],
        "dtype": [num_only_df[col].dtype for col in num_only_df.columns],
    },
    index=num_only_df.columns,
)
unique_values

The numbers encode independent values with no numeric relationship: veterans benefits, own business, and the two the recodes.

One might consider adding "num persons worked for employer" and "year" as well, given the concentration to only a few unique values.

In [0]:
train_df["num persons worked for employer"].value_counts(normalize=True)

In [0]:
train_df["year"].value_counts(normalize=True)

Let's convert these and process with the rest of the categorical columns:

In [0]:
num_to_object_cols = [
    "detailed industry recode", 
    "detailed occupation recode",
    "own business or self employed",
    "veterans benefits",
    "year",
    "num persons worked for employer",
]

In [0]:
for col in num_to_object_cols:
    train_df[col] = train_df[col].astype("object")

Look at high level stats:

In [0]:
num_cols = train_df.select_dtypes('number')
num_cols.describe()

We see some major skew in these fields and some curious values, such as the 9999 and 99999 for the dollar values. 

In [0]:
def plot_numeric(column: str) -> None:
    fig = px.histogram(train_df, x=column, title=f"{column} distribution")
    fig.show()
    
for col in num_cols.columns:
    plot_numeric(col)
    
    

Observations:
- weeks worked is mostly 0 or 52
- most distributions highly skewed to the right with a few large outliers (wage per hour, cap loss/gain, dividend, instance weight)
- age distribution is interestingly bimodal with moderate right skew

It is likely that either 9999 was used as a filler value in these columns or used as an artificial maximum to protect the privacy of some outlier data subjects.

In [0]:
(train_df['wage per hour'] == 9999).sum()

In [0]:
(train_df['capital gains'] == 99999).sum()

In [0]:
(train_df['dividends from stocks'] == 99999).sum()

These are extreme outlier values of the fields (based on the percentiles above). We also saw that there are a really high % of values falling exactly at 0:

In [0]:
for col in num_cols.columns:
    fig = px.histogram(train_df.loc[train_df[col] > 0], x=col)
    fig.show()

This allows us to see the distribution better.

**Proposal**
- we will focus on tree-based algorithms for modelling which are quite robust to outliers and scale differences so for now we'll leave the columns as is during preprocessing

In the future, we can apply log-transformation, scaling to these fields if the models require. We can alternatively consider binning to create categorical fields from these variables too (based on uniform or percentile bins).

### Categorical fields

Lets consider now the categorical fields.

In [0]:
pyo.init_notebook_mode()

TARGET = 'income'


def plot_categories(column: str) -> None:
    """
    Plot top-20 value counts.
    """
    counts = train_df[column].value_counts(normalize=True).head(20)
    fig = px.bar(counts, title=f"{column} distribution")
    fig.show()
    
plot_categories(TARGET)

We have a highly imbalance distribution, with only 6% of train samples in the +50K category. This will likely need addressing, the classification models will be affected and in general, we expect worse performance on this minority class.

Lets see the rest of the columns:

In [0]:
pyo.init_notebook_mode()

for col in train_df.select_dtypes("object"):
    plot_categories(col)

We see most columns are again imbalanced with a high number of unique values and long tails.

In [0]:
train_df.select_dtypes("object").nunique()

If all columns are dummy encoded we get > 450 cols (once one of each category dropped):

In [0]:
train_df.select_dtypes("object").nunique().sum() - train_df.shape[1]

In terms of encoding and processing:
- we would typically create one-hot vectors from the categories to be used with modelling
- to avoid extremely high dim data, we should roll up categories before dummy encoding
    - this can be done based on our insight of related categories,
    - or using automated feature selection after getting each dummy column
- some categories (education) show clear order which we expect also to have a natural correlation with the target (higher education -> higher income) - it would be an option to encode along a rank, say 1 to 5 the level of education.

**Proposal**:
- as our initial approach, we shall dummy encode the categorical fields
- use category frequency thresholds to decide on if that value deserves it's own field
- limit the max number of categories introduced

Our preprocessing pipeline will implement these steps.

## Missing Values 

We see there is only a single column with missing values, for <0.5% of the rows:

In [0]:
train_df.isna().mean()

In [0]:
train_df.isna().sum().sum()

From the data glossary provided:

>Hispanic Origin
Persons of Hispanic origin in this file are determined on
the basis of a question asking if the person is Spanish,
Hispanic, or Latino. If the response is “yes,” a follow-up
question determines a specific ethnic origin, asking to
select their (the person’s) origin from a “flash card”
listing. The flash-card selections are Mexican, Mexican-
American, Chicano, Puerto Rican, Cuban, Cuban
American, or some other Spanish, Hispanic, or Latino
group.

If these values are missing at random, dropping them likely makes little difference to the modelling.

In [0]:
train_df.loc[train_df.isnull().any(axis=1), "race"].value_counts(normalize=True)

In [0]:
train_df['race'].value_counts(normalize=True)

In [0]:
train_df['hispanic origin'].value_counts(normalize=True)

In [0]:
train_df.loc[train_df.isnull().any(axis=1), "income"].value_counts(normalize=True)

The rows do not seem to be skewed with respect to our target or the 'race' column. However, in general we try to avoid dropping rows at all costs to avoid biasing the data inadvertently.

**Proposal**: Since the `hispanic origin` column already has a `Do not know` value, we can fill with that.

## Duplicate Rows

Lets see how many duplicate rows we have:

In [0]:
train_df.duplicated().sum()

This is already significant, but if we look at without our instance weight column:

In [0]:
train_df.drop(columns=['instance weight', 'income']).duplicated().sum()

We can see that without our weight and income column, we get the exact number of duplicate instances mentioned in the data dictionary.

> Number of instances data = 199523
    Duplicate or conflicting instances : 46716
 Number of instances in test = 99762
    Duplicate or conflicting instances : 20936

It is unclear where these are coming from and there is no ID to identify data subjects (as per the anonymization). Note that this is a significant number of rows to drop from our data and it could be that some of these duplicates are legitimate (two people with the same recorded attributes).

In [0]:
without_duplicates = train_df.drop_duplicates(subset=train_df.drop(columns=['instance weight', 'income']).columns)
without_duplicates.shape

Note: the Census does put a fair effort into deduplication. https://www.census.gov/newsroom/blogs/random-samplings/2021/04/how_we_unduplicated.html

>There are several reasons for duplicates in a census:
    We receive more than one response for an address.
    People are counted in more than one place because of potentially complex living situations.
    There might be an issue with the address — a housing unit is on our address list more than once or census materials are misdelivered.
    We use a special algorithm to resolve the first situation and a series of steps to resolve the second and third.

**Proposal**: we will drop these rows since there is no additional evidence that these are not corrupted (see data dict). 

For our tree learning algorithm, having duplicate rows would amount to weighting the data points which is really done through the instance weight column already, which we decided not to use for modelling. In real business situations, we'd explore the reason for duplication and make the best decision based on more in-depth understanding.

## Relationship to Target

We'd like to get a sense which columns are most strongly related to the target. We can do this by standard statistics and tests (chi2 or f-score). We leave the feature selection to the preprocessing notebook and explore select columns here that are promising predictors.

Only weak relationship between the numeric features.

In [0]:
fig = px.imshow(
    train_df.select_dtypes('number').corr(),
    text_auto=True,
    color_continuous_scale='RdBu_r',
    zmin=-1,
    zmax=1
)
fig.show()

Between our target and the numeric features:

In [0]:
pyo.init_notebook_mode()

for col in train_df.select_dtypes('number'):
    fig = px.box(train_df, x=col, color="income")
    fig.show()

After a bit of transformations, we can get a better visual:

In [0]:
pyo.init_notebook_mode()

for col in train_df.select_dtypes('number'):
    filtered = train_df[train_df[col] > 0] # Note there are lot of data points filtered
    filtered[col] = np.log1p(filtered[col])
    fig = px.box(filtered, x=col, color="income")
    fig.show()

Observations:
- large earners are older on average
- we can see some expected relationship with wage and gains/losses as well as weeks worked

We could use a two-sample t-test for checking statistical significance of the difference in distributions (although need to be careful with the large sample size).

Finally, looking at our categorical fields, we can check pivot tables and chi2 scores, selecting the most significant results by a 0.001 p-value threshold to account for multiple testing:

In [0]:
from scipy.stats import chi2_contingency

# run on a sample, chi2 will give more false pos with super large samples
sample = train_df.sample(1000)

results = []
for col in train_df.select_dtypes('object').drop(columns=TARGET):
    contingency_table = sample.groupby([col, TARGET])[TARGET].count().unstack().fillna(0)
    chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)
    results.append(
        {
            'col': col,
            'chi2': chi2_stat,
            'p_value': p_value,
        }
    )
    
chi2_results = pd.DataFrame(results)

# significant with 0.1% cutoff to account for multiple testing
chi2_results[chi2_results["p_value"] < 0.001]

We can see that these are reasonable columns, industry, class of worker, employment status, etc are highly likely to affect the income. We expect these to show up in our modelling as well and help guide prioritizing feature selection.